In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [2]:
import boto3


aws_access_key_id='****'
aws_secret_access_key='****'
# Configured aws credentials
session = boto3.Session(
    aws_access_key_id='****',
    aws_secret_access_key='****',
    region_name='us-west-1'
)

# Created SageMaker client
s3_client = session.client('s3')
bucket_name = 'episafe-data'

In [3]:
# data fetching using s" client connection 
objects = s3_client.list_objects(Bucket=bucket_name)

In [4]:
objects = []
paginator = s3_client.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket=bucket_name)

for page in page_iterator:
    if 'Contents' in page:
        objects.extend(page['Contents'])

In [5]:
# Create a dictionary to organize objects by their folder structure
folders_list = []

# Iterate through objects and organize them into the folder_structure dictionary
for obj in objects[2:]:
    object_key = obj['Key']
    substring = object_key[6:15]
    folders_list.append(substring) 

In [6]:
# Using set to get unique values
unique_values_set = set(folders_list)

# Converting set back to a list
unique_values_list = list(unique_values_set)

In [11]:
for folder in unique_values_list:
    if folder.split('/')[0] == '1904v' or folder == '1904v' or folder == 'train_lab':
        unique_values_list.remove(folder)

In [12]:
different_folders = []
for i in [1110,1869,1876,1904]:
    listt = []
    for j in range(len(unique_values_list)):
        if int(unique_values_list[j].split('/')[0]) == i:
            listt.append(unique_values_list[j])
    different_folders.append(listt)

In [13]:
import io
s3_file_key = 'train/train_labels.csv'

try:
    # Use the S3 client to download the file
    response = s3_client.get_object(Bucket=bucket_name, Key=s3_file_key)
    data = response['Body'].read().decode('utf-8')

    # Read the CSV data using pandas
    labels = pd.read_csv(io.StringIO(data))
    labels2 = pd.read_csv(io.StringIO(data))

    #'df' contains the CSV data as a pandas DataFrame
    print(labels.head()) 

except Exception as e:
    print(f"Error reading CSV from S3: {str(e)}")

                                   filepath  label
0  1110/000/UTC-2020_02_26-23_10_00.parquet      0
1  1110/001/UTC-2020_02_27-17_30_00.parquet      0
2  1110/001/UTC-2020_02_27-17_40_00.parquet      0
3  1110/002/UTC-2020_02_27-17_50_00.parquet      0
4  1110/002/UTC-2020_02_27-18_00_00.parquet      0


In [14]:
labels.set_index('filepath', inplace=True)

In [15]:
files_per_patient = []
for j in range(len(different_folders)):
    files_per_folder = []
    for file in different_folders[j]:
        local_list = []
        for obj in objects[2:]:
            object_key = obj['Key']
            substring = object_key[6:15]
            if substring == file:
                local_list.append(obj['Key'])
        files_per_folder.append(local_list)
    files_per_patient.append(files_per_folder)

In [16]:
import itertools
result1 = list(itertools.chain.from_iterable(files_per_patient))

In [17]:
result2 = list(itertools.chain.from_iterable(result1))

In [18]:
len(result2)

67118

In [19]:
labels

,label
filepath,
1110/000/UTC-2020_02_26-23_10_00.parquet,0
1110/001/UTC-2020_02_27-17_30_00.parquet,0
1110/001/UTC-2020_02_27-17_40_00.parquet,0
1110/002/UTC-2020_02_27-17_50_00.parquet,0
1110/002/UTC-2020_02_27-18_00_00.parquet,0
...,...
2002/102/UTC-2021_03_12-03_30_00.parquet,0
2002/102/UTC-2021_03_12-03_40_00.parquet,0
2002/102/UTC-2021_03_12-03_50_00.parquet,0


In [20]:
#selecting positive files 
positive_files = []

for i in range(labels.shape[0]):
    if labels2['label'][i] == 1 and 'train/'+labels2['filepath'][i] in result2:
        positive_files.append('train/'+labels2['filepath'][i])


In [21]:
len(positive_files)

201

In [22]:
negative_files = []

for i in range(1100):
    if labels2['label'][i] == 0 and 'train/'+labels2['filepath'][i] in result2:
        negative_files.append('train/'+labels2['filepath'][i])

In [23]:
print(len(positive_files))
print(len(negative_files))

201
1093


In [24]:
positive_files[:2]

['train/1110/011/UTC-2020_03_08-11_10_00.parquet',
 'train/1110/011/UTC-2020_03_08-11_20_00.parquet']

In [25]:
negative_files[:2]

['train/1110/000/UTC-2020_02_26-23_10_00.parquet',
 'train/1110/001/UTC-2020_02_27-17_30_00.parquet']

In [26]:
training_p_files = positive_files[:400]
training_n_files = negative_files[0:600]
training_files = training_p_files + training_n_files

In [27]:
for file in training_files:
    if file in ['train/1869/109/UTC-2020_03_16-02_30_00.parquet','train/1869/109/UTC-2020_03_16-02_40_00.parquet','/train/1869/109/UTC-2020_03_16-02_10_00.parquet','train/1869/109/UTC-2020_03_16-02_10_00.parquet','/train/1869/109/UTC-2020_03_16-02_20_00.parquet','/train/1904/024/UTC-2020_05_03-06_40_00.parquet','train/1869/109/UTC-2020_03_16-02_20_00.parquet']:
        training_files.remove(file)

In [28]:
len(training_files)

801

In [30]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [31]:
def change_frequency(df):
    df['timestamp'] = pd.to_datetime(df['utc_timestamp'], unit='s')

    # Set the timestamp column as the DataFrame index
    df.set_index('timestamp', inplace=True)

    # Create a new DataFrame with a datetime index at 4 Hz intervals
    resampled_df = pd.DataFrame(index=pd.date_range(start=df.index.min(), end=df.index.max(), freq='250L'))

    # Use groupby to resample the original data to 4 Hz (128 Hz / 32 = 4 Hz)
    for column in df.columns:
        resampled_df[column] = df[column].groupby(pd.Grouper(freq='250L')).mean()
    
    resampled_df['date_time'] = resampled_df.index
    resampled_df['timestamp_column'] = resampled_df['date_time'].apply(lambda x: pd.Timestamp(x).timestamp())
    resampled_df.reset_index(inplace=True)
    resampled_df.drop(['utc_timestamp','date_time','index'],axis=1,inplace=True)
    return resampled_df


In [32]:
def load_and_preprocess_data(training_files,labels):
    data = []
    local_labels = []
    for j in range(0,len(training_files)):
        file_key = training_files[j]
        print(file_key)
        # Read the Parquet file from S3
        s3_uri = f's3://{bucket_name}/{file_key}'
        df = pd.read_parquet(s3_uri, storage_options={'key': aws_access_key_id, 'secret': aws_secret_access_key})
        missing_values = df.isnull().sum()
       
        # Check if any column has missing values
        if missing_values.any():
            print("The DataFrame contains null values.")
        else:
            df2 = change_frequency(df)
            df2.drop('bvp',inplace=True,axis=1)
            data.append(df2)
            print(file_key[6:])
            print(labels.loc[file_key[6:],'label'])
            local_labels.append(labels.loc[file_key[6:],'label'])
    return np.array(data),np.array(local_labels)

In [33]:
sequences,seq_labels = load_and_preprocess_data(training_files,labels)

train/1110/011/UTC-2020_03_08-11_10_00.parquet


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


1110/011/UTC-2020_03_08-11_10_00.parquet
1
train/1110/011/UTC-2020_03_08-11_20_00.parquet
1110/011/UTC-2020_03_08-11_20_00.parquet
1
train/1110/011/UTC-2020_03_08-11_30_00.parquet
1110/011/UTC-2020_03_08-11_30_00.parquet
1
train/1110/011/UTC-2020_03_08-11_40_00.parquet
1110/011/UTC-2020_03_08-11_40_00.parquet
1
train/1110/011/UTC-2020_03_08-11_50_00.parquet
1110/011/UTC-2020_03_08-11_50_00.parquet
1
train/1110/011/UTC-2020_03_08-12_00_00.parquet
1110/011/UTC-2020_03_08-12_00_00.parquet
1
train/1110/016/UTC-2020_03_17-09_40_00.parquet
1110/016/UTC-2020_03_17-09_40_00.parquet
1
train/1110/016/UTC-2020_03_17-09_50_00.parquet
1110/016/UTC-2020_03_17-09_50_00.parquet
1
train/1110/016/UTC-2020_03_17-10_00_00.parquet
1110/016/UTC-2020_03_17-10_00_00.parquet
1
train/1110/016/UTC-2020_03_17-10_10_00.parquet
1110/016/UTC-2020_03_17-10_10_00.parquet
1
train/1110/016/UTC-2020_03_17-10_20_00.parquet
1110/016/UTC-2020_03_17-10_20_00.parquet
1
train/1110/016/UTC-2020_03_17-10_30_00.parquet
1110/016/U

In [34]:
sequences[0]

array([[-9.80716901e+00,  1.00043703e+01,  6.19991799e+01, ...,
         4.83195705e+01,  3.56598101e+01,  1.58366580e+09],
       [-9.99526442e+00,  1.00043703e+01,  6.19991799e+01, ...,
         4.82721371e+01,  3.56598101e+01,  1.58366580e+09],
       [-9.99526442e+00,  1.00043703e+01,  6.19991799e+01, ...,
         4.82249247e+01,  3.56598101e+01,  1.58366580e+09],
       ...,
       [-4.39994140e+01, -4.69996243e+01,  1.00373954e+00, ...,
         5.20242029e+01,  3.56084359e+01,  1.58366640e+09],
       [-4.39994140e+01, -4.69996243e+01,  1.00373954e+00, ...,
         5.20541204e+01,  3.55806039e+01,  1.58366640e+09],
       [-4.39994140e+01, -4.69996243e+01,  1.12830658e+00, ...,
         5.20864225e+01,  3.55698832e+01,  1.58366640e+09]])

In [35]:
print(len(sequences))
print(len(seq_labels))

734
734


In [36]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(sequences, seq_labels, test_size=0.25, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)


In [37]:
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
X_val_array = np.array(X_val)

In [40]:
epochs = 10
batch_size = 32

In [41]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.1) 
auc_metric = tf.keras.metrics.AUC()

model = tf.keras.Sequential()
model.add(LSTM(units=128,return_sequences=True,input_shape=(2400,8)))
model.add(Dropout(0.2)) 
model.add(LSTM(units=128,return_sequences=True))
model.add(Dropout(0.2)) 
model.add(LSTM(units=128,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='LeakyReLU'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [42]:
history = model.fit(X_train_array, y_train, validation_data=(X_val_array, y_val), epochs=epochs, batch_size=batch_size,verbose=2)

Epoch 1/10
18/18 - 23s - loss: 5.4578 - accuracy: 0.6382 - val_loss: 4.9113 - val_accuracy: 0.7500 - 23s/epoch - 1s/step
Epoch 2/10
18/18 - 5s - loss: 1.2048 - accuracy: 0.7055 - val_loss: 1.7957 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 3/10
18/18 - 5s - loss: 0.8689 - accuracy: 0.7400 - val_loss: 0.5726 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 4/10
18/18 - 5s - loss: 0.5588 - accuracy: 0.7964 - val_loss: 0.8592 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 5/10
18/18 - 5s - loss: 0.6623 - accuracy: 0.7236 - val_loss: 0.6634 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 6/10
18/18 - 5s - loss: 0.6189 - accuracy: 0.7782 - val_loss: 0.9229 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 7/10
18/18 - 5s - loss: 0.6241 - accuracy: 0.8036 - val_loss: 0.5625 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 8/10
18/18 - 5s - loss: 0.5289 - accuracy: 0.8145 - val_loss: 0.5649 - val_accuracy: 0.7500 - 5s/epoch - 262ms/step
Epoch 9/10
18/18 - 5s - l

In [43]:
#Saving the model 
model.save('lstm_model_v2_08145.h5')

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Specify your S3 bucket name and the S3 object key
bucket_name = 'lstm-artifacts'
s3_object_key = '/lstm_model_v2_06847.h5'

# Upload the local model file to S3
s3.upload_file('lstm_model_v2_06847.h5', bucket_name, s3_object_key)

print(f"Model file saved to S3://{bucket_name}/{s3_object_key}")


Model file saved to S3://lstm-artifacts//lstm_model_v2_06847.h5


In [1]:
import tensorflow as tf

# Load the saved TensorFlow model
model = tf.keras.models.load_model("lstm_model_v2_081.h5")

# Create a TFLite converter with custom options
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.experimental_new_converter = True  # This flag enables a new converter.

# Convert the model to TFLite format
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("episafe_lstm_model_081.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp8c6sscn0/assets


INFO:tensorflow:Assets written to: /tmp/tmp8c6sscn0/assets


In [7]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="episafe_lstm_model.tflite")

# Allocate memory for input and output tensors
interpreter.allocate_tensors()

# Get details about the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print information about input and output tensors
print("Input tensor details:")
print(input_details)

print("\nOutput tensor details:")
print(output_details)

#  Information about the model's input and output shapes and data types
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']
input_dtype = input_details[0]['dtype']
output_dtype = output_details[0]['dtype']

print("\nInput tensor shape:", input_shape)
print("Input tensor data type:", input_dtype)
print("Output tensor shape:", output_shape)
print("Output tensor data type:", output_dtype)


Input tensor details:
[{'name': 'serving_default_lstm_input:0', 'index': 0, 'shape': array([    1, 76800,     9], dtype=int32), 'shape_signature': array([   -1, 76800,     9], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Output tensor details:
[{'name': 'StatefulPartitionedCall:0', 'index': 82, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Input tensor shape: [    1 76800     9]
Input tensor data type: <class 'numpy.float32'>
Output tensor shape: [1 1]
Output tensor data type: <class 'numpy.float32'>
